In [10]:
import pandas as pd
import numpy as np

# read the CSV file
file_path = 'forecast_history.csv' 
data = pd.read_csv(file_path)

# clean and format data
data = data.replace({'%': '', '\$': '', ',': '', 'I': '1', 'O': '0'}, regex=True)
data = data.apply(pd.to_numeric, errors='coerce')

# remove outliers using IQR method
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)  
    IQR = Q3 - Q1 
    lower_bound = Q1 - 1.5 * IQR  
    upper_bound = Q3 + 1.5 * IQR 
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# remove outliers
for forecaster in ['Westpac: 4 year forecast', 'Joe Bloggs: 2 year forecast', 'Harry Spent: 5 year forecast']:
    data = remove_outliers(data, forecaster)
    
# calculate actual percentage changes in median house price year over year
data['Actual_Change'] = data['Median house price'].pct_change() * 100

# function to calculate MAPE
def calculate_mape(actual, forecast):
    return np.mean(np.abs((actual - forecast) / actual)) * 100

results = {}
for forecaster in ['Westpac: 4 year forecast', 'Joe Bloggs: 2 year forecast', 'Harry Spent: 5 year forecast']:
    data[f'{forecaster}_Error'] = np.abs(data['Actual_Change'] - data[forecaster])
    results[forecaster] = calculate_mape(data['Actual_Change'][1:], data[forecaster][1:])

# final results
for forecaster, mape in results.items():
    print(f"{forecaster} MAPE: {mape:.2f}%")


    Unnamed: 0  Median house price  Westpac: 4 year forecast  \
0         2011              340000                      56.0   
1         2012              370000                      53.0   
2         2013              350000                       NaN   
3         2014              420000                      13.0   
4         2015              425000                      33.0   
5         2016              500000                      -5.0   
6         2017              520000                      45.0   
7         2018              550000                      34.0   
8         2019              596000                      34.0   
9         2020              610000                      20.0   
10        2021              660000                     -20.0   
11        2022              720000                      15.0   
12        2023               73000                      15.0   
13        2024              760000                     -10.0   

    Joe Bloggs: 2 year forecast  Harry 